In [5]:
import torch
import pandas as pd
import ltn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt


## Data Preparation

In [2]:
s_data = pd.read_csv('src\data\Stud_E-mobility_data_staticLimit.csv')

In [3]:
s_data = s_data[['_time','GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT',
      #  'DEMAND_LIMIT_INDICATOR', 
       'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'BATTERY_CHARGED_ENERGY', 'BATTERY_DISCHARGED_ENERGY', 'PV_POWER',
       'PV_ENERGY'
    ]]

In [8]:
def label_charging(row):
    if row["BATTERY_SOC"] > 80:
        return "Fully Covered by Local Battery"
    elif 40 <= row["BATTERY_SOC"] < 80:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    elif 15 <= row["BATTERY_SOC"] <= 40:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    elif row["BATTERY_SOC"] < 15:
        return "Battery Discharge Stopped due to Battery Health"
    else:
        print(row["BATTERY_SOC"])
        print(row["GARAGE_EXTERNAL_POWER"])
        return "Unknown"

# Apply the labeling function to create the new column "DRAWN_FROM"
s_data["DRAWN_FROM"] = s_data.apply(label_charging, axis=1)

In [9]:
s_data_small = s_data.sample(frac=0.4, random_state=42)

In [10]:
features = s_data.drop(['_time','DRAWN_FROM', 'BATTERY_DISCHARGE_POWER', 'BATTERY_CHARGED_ENERGY',  'BATTERY_DISCHARGED_ENERGY', 'GARAGE_EXTERNAL_POWER'], axis=1)
target = s_data['DRAWN_FROM']

In [8]:
# calculate number of points in each class
print(target.value_counts())

DRAWN_FROM
Battery Charged from Grid                          54783
Partially Covered by Local Battery                  4457
Battery Discharge Stopped due to Battery Health      202
Name: count, dtype: int64


# Logic Tensor Networks


In [11]:

encoder = LabelEncoder()
en_targ = encoder.fit_transform(target)

In [12]:
# Get unique values of the target
unique_values = np.unique(en_targ)
unique_values

array([0, 1, 2])

In [13]:

features_train, features_test, target_train, target_test = train_test_split(features, en_targ, test_size=0.2, random_state=42)

In [14]:
features_train = torch.tensor(features_train.to_numpy()).float()
features_test = torch.tensor(features_test.to_numpy()).float()


In [15]:
l_A = ltn.Constant(torch.tensor([1, 0, 0]))
l_B = ltn.Constant(torch.tensor([0, 1, 0]))
l_C = ltn.Constant(torch.tensor([0, 0, 1]))

In [16]:
class MLP(torch.nn.Module):
    def __init__(self, layer_sizes=(4, 100, 52, 52, 3)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits

class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)
        return out


In [17]:
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))

# we define the connectives, quantifiers, and the SatAgg
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

### Utils

In [18]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.unique_labels = np.unique(labels) 

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        # Create a list of indices for each class
        indices_per_class = {label: np.where(self.labels == label)[0] for label in self.unique_labels}

        # Calculate the number of samples per class in each batch
        samples_per_class = self.batch_size // len(self.unique_labels)

        for _ in range(len(self)):
            batch_indices = []

            for label in self.unique_labels:
                # Randomly sample indices for this class
                class_indices = np.random.choice(indices_per_class[label], size=samples_per_class, replace=True)
                batch_indices.extend(class_indices)

            # If the batch size is not a multiple of the number of classes, fill the rest of the batch randomly
            if len(batch_indices) < self.batch_size:
                extra_indices = np.random.choice(np.arange(len(self.labels)), size=self.batch_size - len(batch_indices))
                batch_indices.extend(extra_indices)

            if self.shuffle:
                np.random.shuffle(batch_indices)

            yield self.data[batch_indices], self.labels[batch_indices]

In [19]:
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[labels == 0])
        x_B = ltn.Variable("x_B", data[labels == 1])
        x_C = ltn.Variable("x_C", data[labels == 2])
        mean_sat += SatAgg(
            Forall(x_A, P(x_A, l_A)),
            Forall(x_B, P(x_B, l_B)),
            Forall(x_C, P(x_C, l_C))
        )
    mean_sat /= len(loader)
    return mean_sat

In [20]:
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = mlp(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)


In [21]:

train_loader = DataLoader(features_train, target_train, 256, shuffle=True)
test_loader = DataLoader(features_test, target_test, 256, shuffle=False)

In [22]:
# get unique values of the target and how much there is of each
unique_values, counts = np.unique(target_train, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([43811,   165,  3577], dtype=int64))

In [22]:
optimizer = torch.optim.Adam(P.parameters(), lr=0.001)

for epoch in range(500):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) # class A examples
        x_B = ltn.Variable("x_B", data[labels == 1]) # class B examples
        x_C = ltn.Variable("x_C", data[labels == 2]) # class C examples
        # print(x_B)
        sat_agg = SatAgg(
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True))
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

 epoch 0 | loss 0.4128 | Train Sat 0.662 | Test Sat 0.665 | Train Acc 0.861 | Test Acc 0.861
 epoch 20 | loss 0.2891 | Train Sat 0.714 | Test Sat 0.722 | Train Acc 0.909 | Test Acc 0.913


KeyboardInterrupt: 

In [ ]:

# Initialize a matrix to store the counts
class_counts = np.zeros((500, 3))

for epoch in range(500):
    for batch_idx, (data, labels) in enumerate(train_loader):
        # Count the number of examples in each class
        class_counts[epoch, 0] = np.sum(labels == 0)
        class_counts[epoch, 1] = np.sum(labels == 1)
        class_counts[epoch, 2] = np.sum(labels == 2)
    
# Convert the counts to a DataFrame
df_class_counts = pd.DataFrame(class_counts, columns=['Class A', 'Class B', 'Class C'])

# Plot the heatmap
sns.heatmap(df_class_counts)
plt.show()

What if we tried to again predict the peak shaving, and check for different predicates what is their satisfaction level.
- it would make sense that if we have a predicate: SOC <= 20 stop peak shaving, than if the treshold is good (or the rule in general), than the satisfaction of this rule will be high.
- This could also allow us to check the relation to the solar panels ? -> if the predicat that includes it does better than that that does not include it


In [24]:
# take 10% of the dataset 
data_small = s_data.sample(frac=0.99, random_state=42)


In [25]:
def check_peak_shaving(row):
    if row['GARAGE_EXTERNAL_POWER'] >= row['DEMAND_LIMIT']:
        return True
    else:
        return False

data_small['Peak_Shaving'] = data_small.apply(check_peak_shaving, axis=1)


In [26]:
# Define the thresholds
battery_discharge_power_threshold = 0
pv_power_threshold = 0.5

def adjust_peak_shaving(row):
    if row['Peak_Shaving'] == True:
        if row['BATTERY_DISCHARGE_POWER'] > battery_discharge_power_threshold or row['PV_POWER'] > pv_power_threshold:
            return True
        else:
            return False
    else:
        return False



In [27]:
data_small['Peak_Shaving'] = data_small.apply(adjust_peak_shaving, axis=1)

In [28]:
features = data_small.drop(['_time','Peak_Shaving','DEMAND_LIMIT', 'BATTERY_CHARGED_ENERGY',  'BATTERY_DISCHARGED_ENERGY'], axis=1)
target = data_small['Peak_Shaving']

In [29]:

# Create a label encoder
le = LabelEncoder()

# Fit the encoder to the 'DRAWN_FROM' column and transform it
features['DRAWN_FROM'] = le.fit_transform(features['DRAWN_FROM'])

In [30]:
features

,GARAGE_EXTERNAL_POWER,BATTERY_SOC,BATTERY_DISCHARGE_POWER,PV_POWER,PV_ENERGY,DRAWN_FROM
35762,1.052765,40.5,-0.232000,0.912828,0.015625,0
5687,1.244171,40.5,-0.317000,0.009138,0.000000,0
56605,26.796896,41.0,-0.422000,0.003416,0.000000,0
34300,49.957039,21.0,46.626003,2.144122,0.046875,0
30633,9.857449,54.5,-0.351000,0.452726,0.003906,0
...,...,...,...,...,...,...
53401,5.742216,40.5,-0.225000,0.007116,0.000000,0
19531,6.316434,40.5,-0.498000,0.007033,0.000000,0
8130,13.589869,41.0,-0.449000,0.003036,0.000000,0
56156,50.148445,33.0,-10.146001,-0.002565,0.000000,2


In [31]:

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [32]:
features_train = torch.tensor(features_train.to_numpy()).float()
target_train = torch.tensor(target_train.to_numpy()).float()
features_test = torch.tensor(features_test.to_numpy()).float()
target_test = torch.tensor(target_test.to_numpy()).float()

Lets try first with a simple rule 
if GARAGE_EXTERNAL_POWER > DEMAND_LIMIT and SOC >= 15%

In [33]:
import ltn
# we define predicate A
class ModelA(torch.nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.sigmoid = torch.nn.Sigmoid()
        self.layer1 = torch.nn.Linear(6, 16)  
        self.layer2 = torch.nn.Linear(16, 16)
        self.layer3 = torch.nn.Linear(16, 1)
        self.elu = torch.nn.ELU()

    def forward(self, x):
        x = self.elu(self.layer1(x))
        x = self.elu(self.layer2(x))
        return self.sigmoid(self.layer3(x))


A = ltn.Predicate(ModelA())

And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

In [34]:
from sklearn.metrics import accuracy_score
import numpy as np

# this is a standard PyTorch DataLoader to load the dataset for the training and testing of the model
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.unique_labels = np.unique(labels) 

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        # Create a list of indices for each class
        indices_per_class = {label: np.where(self.labels == label)[0] for label in self.unique_labels}

        # Calculate the number of samples per class in each batch
        samples_per_class = max(1, self.batch_size // len(self.unique_labels))

        for _ in range(len(self)):
            batch_indices = []

            for label in self.unique_labels:
                # Randomly sample indices for this class
                class_indices = np.random.choice(indices_per_class[label], size=samples_per_class, replace=True)
                batch_indices.extend(class_indices)

            # If the batch size is not a multiple of the number of classes, fill the rest of the batch randomly
            if len(batch_indices) < self.batch_size:
                extra_indices = np.random.choice(np.arange(len(self.labels)), size=self.batch_size - len(batch_indices))
                batch_indices.extend(extra_indices)

            if self.shuffle:
                np.random.shuffle(batch_indices)

            yield self.data[batch_indices], self.labels[batch_indices]


# define metrics for evaluation of the model

# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[torch.nonzero(labels)])  # positive examples
        x_not_A = ltn.Variable("x_not_A",
                               data[torch.nonzero(torch.logical_not(labels))])  # negative examples
        mean_sat += SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )
    mean_sat /= len(loader)
    return mean_sat

# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = A.model(data).detach().numpy()
        predictions = np.where(predictions > 0.5, 1., 0.).flatten()
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

# create train and test loader, 50 points each
# batch size is 64, meaning there is only one batch for epoch
train_loader = DataLoader(features_train, target_train, 1024, True)
test_loader = DataLoader(features_test, target_test, 1024, False)

In [ ]:
optimizer = torch.optim.Adam(A.parameters(), lr=0.001)

# training of the predicate A using a loss containing the satisfaction level of the knowledge base
# the objective it to maximize the satisfaction level of the knowledge base
for epoch in range(500):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels==1]) # positive examples
        x_not_A = ltn.Variable("x_not_A", data[labels==0]) # negative examples
        sat_agg = SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 1 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

## Rules satisfaction

Axioms that are tried

- for every instance if peak shaving is true than Battery_SOC > 15:

In [35]:
def compute_sat_level_phi(loader, phi):
    sat_values = []
    for features, labels in loader:
        for feature, label in zip(features, labels):
            sat_values.append(phi(feature, label).value)
    mean_sat = torch.mean(torch.stack(sat_values))
    return mean_sat


In [43]:
features

,GARAGE_EXTERNAL_POWER,BATTERY_SOC,BATTERY_DISCHARGE_POWER,PV_POWER,PV_ENERGY,DRAWN_FROM
35762,1.052765,40.5,-0.232000,0.912828,0.015625,0
5687,1.244171,40.5,-0.317000,0.009138,0.000000,0
56605,26.796896,41.0,-0.422000,0.003416,0.000000,0
34300,49.957039,21.0,46.626003,2.144122,0.046875,0
30633,9.857449,54.5,-0.351000,0.452726,0.003906,0
...,...,...,...,...,...,...
53401,5.742216,40.5,-0.225000,0.007116,0.000000,0
19531,6.316434,40.5,-0.498000,0.007033,0.000000,0
8130,13.589869,41.0,-0.449000,0.003036,0.000000,0
56156,50.148445,33.0,-10.146001,-0.002565,0.000000,2


In [74]:
# GARAGE_EXTERNAL_POWER > 50

def P(tensor):
    return tensor >= 50

# Wrap the predicate in an LTN Predicate
P = ltn.Predicate(None, P)

In [38]:
# peak shaving is True
def Q(tensor):
    return tensor >= 0.5  # returns True if p_shav is True

# Wrap the predicate in an LTN Predicate
Q = ltn.Predicate(None, Q)

In [75]:

def phi(features, labels):

    features = features[0].view(-1, 1)
    labels = labels.view(-1, 1)
    # Create a variable that represents the data
    p = ltn.Variable("p", features)
    q = ltn.Variable("q", labels)


    # Return the satisfaction degree of the formula

    return Forall(p, Implies(P(p), Q(q)))

In [76]:
compute_sat_level_phi(test_loader, phi)

tensor(0.9926)

In [77]:
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())

def PeakShaving(tensor):
    return tensor >= 0.5  # returns True if peak shaving is True

PeakShaving = ltn.Predicate(None, PeakShaving)

def Battery_SOC_Greater_15(tensor):
    return tensor > 35  # returns True if Battery_SOC > 15

Battery_SOC_Greater_15 = ltn.Predicate(None, Battery_SOC_Greater_15)

def Battery_SOC_small(tensor):
    return tensor < 45  # returns True if Battery_SOC > 15

Battery_SOC_small = ltn.Predicate(None, Battery_SOC_small)

def phi(features, label):
   
    label = label.view(-1, 1)
    sc = features[1].view(-1,1)
    # x = ltn.Variable("x", features)
    y = ltn.Variable("y", label)
    s = ltn.Variable("s", sc)
    return Forall([y, s], Implies(PeakShaving(y), And(Battery_SOC_Greater_15(s), Battery_SOC_small(s))), p=5)




In [78]:
compute_sat_level_phi(test_loader, phi)

tensor(0.7524)

What could be interesting is more general rules, that don't require the tresholds
- for all x if peak shaving than energy is given from both sources 

In [79]:
# energy comes from the power plant and the battery
def gives_energy(tensor):
    # print(tensor)
    return tensor > 0

gives_energy = ltn.Predicate(None, gives_energy)

def phiComp(features, label):
    label = label.view(-1, 1)
    bp = features[2].view(-1,1) #batery power
    pp = features[3].view(-1,1) #batery power
    # x = ltn.Variable("x", features)
    y = ltn.Variable("y", label)
    b = ltn.Variable("b", bp)
    p = ltn.Variable("p", pp)
    return Forall([y, b, p], Implies(PeakShaving(y), And(gives_energy(b), gives_energy(p))), p=5)


In [80]:
compute_sat_level_phi(test_loader, phiComp)

tensor(0.7871)

In [81]:
# energy comes from the power plant and the battery
def gives_energy(tensor):
    # print(tensor)
    return tensor > 0

def gets_charged(tensor):
    # print(tensor)
    return tensor < 0

gives_energy = ltn.Predicate(None, gives_energy)
gets_charged = ltn.Predicate(None, gets_charged)

def phit2(features, label):
    label = label.view(-1, 1)
    bp = features[2].view(-1,1) #batery power
    pp = features[3].view(-1,1) #batery power
  
    b = ltn.Variable("b", bp)
    p = ltn.Variable("p", pp)
    return Forall([b, p], Implies(gives_energy(p), gives_energy(b)), p=5)


In [82]:
compute_sat_level_phi(train_loader, phit2)

tensor(0.4665)

In [83]:

optimizer = torch.optim.Adam(A.parameters(), lr=0.001)

# training of the predicate A using a loss containing the satisfaction level of the knowledge base
# the objective it to maximize the satisfaction level of the knowledge base
for epoch in range(500):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels==1]) # positive examples
        x_not_A = ltn.Variable("x_not_A", data[labels==0]) # negative examples
        # p_shav = labels
        sat_agg = SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A))),
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f | Phi Sat %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader), compute_sat_level_phi(test_loader, phit2)))

 epoch 0 | loss 0.3080 | Train Sat 0.741 | Test Sat 0.739 | Train Acc 0.918 | Test Acc 0.923 | Phi Sat 0.487


KeyboardInterrupt: 

Plan for now: 
- check whith a more complicated new rule how does the satisfaction work 

Idea:
- to learn the treshold we need to save them as variables and create another Predicate to optimze them. But what do we base the optimization on? 

What can we use for loss? Assuming we want to learn tresholds, what would we use to validate how good we do on the data? 
- Maybe based on my simple labeled peak shaving? -- For which values is the peak shaving true. Should this be done with another NN model? I guess we save the tresholds as predicates, similarly as I did with Battery_SOC_Greater_15, but the treshold has to be learnable. 

### Model that guesses peak shaving based on the ground truth

For what SOC
- e-cars charging is completely covered by the local battery
- e-cars charging power is covered by local battery.
- local battery is charged from the grid.
- Battery discharging is stopped due to battery health


In [84]:
ds = pd.read_csv('src\data\Stud_E-mobility_data_staticLimit.csv')

In [85]:
ds["POWER_DEMAND"] = ds["WALLBOX_1_POWER"] + ds["WALLBOX_2_POWER"] + ds["WALLBOX_3_POWER"] + ds["WALLBOX_A_POWER"] + ds["WALLBOX_B_POWER"] + ds["WALLBOX_C_POWER"] + ds["WALLBOX_FASTCHARGER_POWER"]
ds["POWER_SUPPLY"] = ds["GARAGE_EXTERNAL_POWER"] + ds["PV_POWER"] + ds["BATTERY_DISCHARGE_POWER"]

e-cars charging is completely covered by the local battery

In [86]:
is_covered_prc = ((ds["BATTERY_DISCHARGE_POWER"] / ds["POWER_SUPPLY"]) >= 0.98).any()

print(f"Does 'BATTERY_DISCHARGE_POWER' ever cover 100% of 'POWER_SUPPLY'? {is_covered_prc}")

Does 'BATTERY_DISCHARGE_POWER' ever cover 100% of 'POWER_SUPPLY'? True


In [87]:
len(ds[ds["BATTERY_DISCHARGE_POWER"] / ds["POWER_SUPPLY"] < 0])

52480

In [88]:
len(ds[ds["BATTERY_DISCHARGE_POWER"] < 0])

52504

In [89]:
def label_row(row):
    if row["BATTERY_DISCHARGE_POWER"] / row["POWER_SUPPLY"] >= 0.95:
        return "completely covered"
    elif row["BATTERY_DISCHARGE_POWER"] > 0.05:
        return "covered"
    elif row["BATTERY_DISCHARGE_POWER"] < 0:
        return "charged"
    else:
        return "stopped"
    

ds["Label"] = ds.apply(lambda row: label_row(row), axis=1)

In [90]:
# give me counts of unique values of labels
ds["Label"].value_counts()

Label
charged               52479
covered                6927
completely covered       27
stopped                   9
Name: count, dtype: int64

In [95]:
def label_row(row):
    if row["BATTERY_DISCHARGE_POWER"] / row["POWER_SUPPLY"] >= 0.8:
        return "completely covered"
    elif row["BATTERY_DISCHARGE_POWER"] > 0.02:
        return "covered"
    else:
        return "charged"
    
    

ds["Label"] = ds.apply(lambda row: label_row(row), axis=1)

In [96]:
# give me counts of unique values of labels
ds["Label"].value_counts()

Label
charged               52485
covered                6929
completely covered       28
Name: count, dtype: int64

Now that we have that, we can try to optimize tresholds for SOC so that they help us predict the Label

In [97]:
soc_tensor = torch.tensor(ds['BATTERY_SOC'].values, dtype=torch.float32)
external_power_tensor = torch.tensor(ds['GARAGE_EXTERNAL_POWER'].values, dtype=torch.float32)
demand_limit_tensor = torch.tensor(ds['DEMAND_LIMIT'].values, dtype=torch.float32)

How should it work? 
- the model should predict the "Label" (from battery SOC?), and than based on how good the prediction with a curent treshold is update the tresholds
- there are 3 tresholds: one for when the battery is covering for all of the grid, one for when the batery is covering some part of the needed energy, pme for whem it is just charging

How do I make the treshold be something that gets learned?
- predicate 1: learning labels from ds based on battery SOC and tresholds 
- predicate 2: learning tresholds based on the labels and battery SOC 

Axioms: 
- for all x (SOC values): Label IMPLIES x in treshold => if the label is correct than the value is in the treshold   

WHAT ACTUALLY SHOULD BE DONE IS TO MAKE A TRESHOLDS PREDICTION NN AND ADD SOME LOGIC RULES THAT SHOULD BE SATISFIED FOR IT

MODEL 1: take the SOC and tresholds and compute labels

In [98]:
class MLP(torch.nn.Module):
    def __init__(self, layer_sizes=(3, 16, 16, 1)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, *x):
        x = list(x)
        if len(x) == 1:
            x = x[0]
        else:
            x = torch.cat(x, dim=1)
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
        out = self.sigmoid(self.linear_layers[-1](x))
        return out
    

Label = ltn.Predicate(MLP([1, 20, 3]))  # 3 output classes: "charged", "covered", "completely covered"

# Define connectives, quantifiers, and SatAgg
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()


MODEL 2: take label and SOC and compute treshold

In [113]:
charged_ds = ds[ds["Label"] == "charged"]
covered_ds = ds[ds["Label"] == "covered"]
completely_covered_ds = ds[ds["Label"] == "completely covered"]

In [193]:
class TMLP(torch.nn.Module):
    def __init__(self, layer_sizes=(3, 16, 16, 1)):
        super(TMLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, *x):
        x = list(x)
        if len(x) == 1:
            x = x[0]
        else:
            x = torch.cat(x, dim=1)
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
        out = self.linear_layers[-1](x)
        return out
    
t1 = ltn.Predicate(TMLP([2, 20, 1])) # output is a treshold, input is a battery soc and label 
t2 = ltn.Predicate(TMLP([2, 20, 1]))  
t3 = ltn.Predicate(TMLP([2, 20, 1]))  

# f = ltn.Function(TMLP())
# alpha = 0.05
tr1 = ltn.Constant(torch.tensor([80]), trainable=True)
tr2 = ltn.Constant(torch.tensor([40]), trainable=True)
tr3 = ltn.Constant(torch.tensor([15]), trainable=True)

# alpha = 0.05
# Eq = ltn.Predicate(func=lambda u,v: torch.exp(-alpha * torch.sqrt(torch.sum(torch.square(u-v), dim=1))))


In [120]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.unique_labels = np.unique(labels) 

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        # Create a list of indices for each class
        indices_per_class = {label: np.where(self.labels == label)[0] for label in self.unique_labels}

        # Calculate the number of samples per class in each batch
        samples_per_class = max(1, self.batch_size // len(self.unique_labels))

        for _ in range(len(self)):
            batch_indices = []

            for label in self.unique_labels:
                # Randomly sample indices for this class
                class_indices = np.random.choice(indices_per_class[label], size=samples_per_class, replace=True)
                batch_indices.extend(class_indices)

            # If the batch size is not a multiple of the number of classes, fill the rest of the batch randomly
            if len(batch_indices) < self.batch_size:
                extra_indices = np.random.choice(np.arange(len(self.labels)), size=self.batch_size - len(batch_indices))
                batch_indices.extend(extra_indices)

            if self.shuffle:
                np.random.shuffle(batch_indices)

            yield self.data[batch_indices], self.labels[batch_indices]


# def compute_sat_level(loader):
#     mean_sat = 0
#     for x_d, y_d in loader:
#         x = ltn.Variable("x", x_d)
#         y = ltn.Variable("y", y_d)
#         mean_sat += Forall(ltn.diag(x,y), Eq(f(x), y)).value
#     mean_sat /= len(loader)
#     return mean_sat

# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = f.model(data).detach().numpy()
        # predictions = np.where(predictions > 0.5, 1., 0.).flatten()
        mean_accuracy += torch.nn.MSELoss(labels, predictions)
    return mean_accuracy / len(loader)



In [103]:

# Encode the labels
le = LabelEncoder()
encoded_labels = le.fit_transform(ds["Label"].values)
label_tensor = torch.tensor(encoded_labels).float()
# label_tensor = torch.tensor(encoded_labels, dtype=torch.float32).view(-1, 1)

# Prepare the data
soc_tensor = torch.tensor(ds['BATTERY_SOC'].values, dtype=torch.float32)

In [104]:
# Create a DataLoader
dataloader = DataLoader(soc_tensor, label_tensor, batch_size=32, shuffle=True)


In [ ]:
# def phi1(features, label):
#     print(features)
#     print(label)
#     label = label.view(-1, 1)
#     # soc = features[0].view(-1, 1)
#     y = ltn.Variable("y", features)
#     s = ltn.Variable("s", label)
#     return Forall([y, s], Implies(Label(y), And(Battery_SOC_Greater_tr1(s), t1(y))), p=5)

In [165]:
# Define the predicates for each threshold
def Battery_SOC_Greater_tr1(tensor):
    return tensor >= tr1.value

Battery_SOC_Greater_tr1 = ltn.Predicate(None, Battery_SOC_Greater_tr1)

def Battery_SOC_Between_tr1_tr2(tensor):
    return ((tensor >= tr2.value) & (tensor < tr1.value)).float()


Battery_SOC_Between_tr1_tr2 = ltn.Predicate(None, Battery_SOC_Between_tr1_tr2)


def Battery_SOC_Less_tr2(tensor):
    return (tensor < tr2.value).float()

Battery_SOC_Less_tr2 = ltn.Predicate(None, Battery_SOC_Less_tr2)


In [ ]:
def map_to_label(tensor):
    if Battery_SOC_Greater_tr1(tensor):
        return 'completely covered'
    elif Battery_SOC_Between_tr1_tr2(tensor):
        return 'covered'
    else:
        return 'charged'

map_to_label = ltn.Function(None, map_to_label)

In [168]:
def placPred(tensor):
    return tensor

placPred = ltn.Predicate(None, placPred)

In [194]:
tr1.value.requires_grad_(True)
tr2.value.requires_grad_(True)
tr3.value.requires_grad_(True)
optimizer = torch.optim.Adam([tr1.value, tr2.value, tr3.value], lr=0.001)

for epoch in range(500):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        i2 = (labels == 2)
        i1 = (labels == 1)
        i0 = (labels == 0)

        l2 = ltn.Variable("l2", labels[i2] / 2)
        l1 = ltn.Variable("l1", labels[i1] / 2)
        l0 = ltn.Variable("l0", labels[i0] / 2)
        p2 = ltn.Variable("p2", data[i2])
        p1 = ltn.Variable("p1", data[i1])
        p0 = ltn.Variable("p0", data[i0])
    
        print(Forall(p2, Implies(Battery_SOC_Greater_tr1(p2), placPred(l2)), p=5))
        print("WORKED")
        sat_agg = SatAgg(
            Forall(p2, Implies(Battery_SOC_Greater_tr1(p2), placPred(l2)), p=5).value.mean(),
            Forall(p1, Implies(Battery_SOC_Between_tr1_tr2(p1), placPred(l1)), p=5).value.mean(),
            Forall(p0, Implies(Battery_SOC_Less_tr2(p0), placPred(l0)), p=5).value.mean()
        )
        
        # Compute the loss
        loss = 1 - sat_agg
        train_loss += loss.item()
        
        # Backpropagate the loss and update the weights
        loss.backward()
        optimizer.step()
    
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {train_loss / len(dataloader)}")

LTNObject(value=tensor([0.9999, 0.9999, 0.9999, 0.9999, 0.9999, 0.9999, 0.9999, 0.9999, 0.9999,
        0.9999]), free_vars=['l2'])
WORKED


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn